In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def build_model(hp):
    input_node = keras.Input(shape=(20,))
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    output_node = layers.Dense(units=units, activation="relu")(input_node)
    output_node = layers.Dense(units=units, activation="relu")(output_node)
    output_node = layers.Dense(units=1, activation="sigmoid")(output_node)
    model = keras.Model(input_node, output_node)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
    return model



In [2]:
from keras_tuner import RandomSearch

tuner = RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=5,
    executions_per_trial=3,
    directory="my_dir",
    project_name="helloworld",
)


In [3]:
tuner.search_space_summary()


Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [4]:
import numpy as np

x_train = np.random.rand(100, 20)
y_train = np.random.rand(100, 1)
x_val = np.random.rand(20, 20)
y_val = np.random.rand(20, 1)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_val, y_val))


Trial 5 Complete [00h 00m 05s]
val_mae: 0.20560526847839355

Best val_mae So Far: 0.20300784707069397
Total elapsed time: 00h 00m 32s


In [5]:
tuner.results_summary(5)


Results summary
Results in my_dir/helloworld
Showing 5 best trials
Trial summary
Hyperparameters:
units: 160
Score: 0.20300784707069397
Trial summary
Hyperparameters:
units: 128
Score: 0.20322332282861075
Trial summary
Hyperparameters:
units: 64
Score: 0.20358007649580637
Trial summary
Hyperparameters:
units: 32
Score: 0.20395702123641968
Trial summary
Hyperparameters:
units: 96
Score: 0.20560526847839355


In [6]:
from tensorflow import keras

best_models = tuner.get_best_models(num_models=2)
best_model = best_models[0]
best_model.save("path_to_best_model")
best_model = keras.models.load_model("path_to_best_model")
print(best_model.predict(x_val))
best_model.summary()


1/1 [==============================] - 0s 92ms/step
[[0.5371855 ]
 [0.5330885 ]
 [0.55809957]
 [0.54377085]
 [0.5166314 ]
 [0.5366102 ]
 [0.5439118 ]
 [0.52639407]
 [0.4927856 ]
 [0.55669546]
 [0.5069383 ]
 [0.5147797 ]
 [0.5213642 ]
 [0.47800633]
 [0.53243273]
 [0.5115892 ]
 [0.49896082]
 [0.4933017 ]
 [0.51752114]
 [0.53747684]]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 dense (Dense)               (None, 160)               3360      
                                                                 
 dense_1 (Dense)             (None, 160)               25760     
                                                                 
 dense_2 (Dense)             (None, 1)                 161       
                                                          

In [7]:

def build_model(hp):
    input_node = keras.Input(shape=(20,))
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    output_node = layers.Dense(units=units, activation="relu")(input_node)
    output_node = layers.Dense(units=units, activation="relu")(output_node)
    output_node = layers.Dense(units=1, activation="sigmoid")(output_node)
    model = keras.Model(input_node, output_node)
    optimizer_name = hp.Choice("optimizer", ["adam", "adadelta"])
    learning_rate = hp.Float(
        "learning_rate", min_value=1e-5, max_value=0.1, sampling="log"
    )
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
    return model


tuner = RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=5,
    executions_per_trial=3,
    directory="my_dir",
    project_name="helloworld",
)


In [8]:
import keras_tuner as kt


class Regressor(kt.HyperModel):
    def build(self, hp):
        input_node = keras.Input(shape=(20,))
        units = hp.Int("units", min_value=32, max_value=512, step=32)
        output_node = layers.Dense(units=units, activation="relu")(input_node)
        output_node = layers.Dense(units=units, activation="relu")(output_node)
        output_node = layers.Dense(units=1, activation="sigmoid")(output_node)
        model = keras.Model(input_node, output_node)
        optimizer_name = hp.Choice("optimizer", ["adam", "adadelta"])
        learning_rate = hp.Float(
            "learning_rate", min_value=1e-5, max_value=0.1, sampling="log"
        )
        if optimizer_name == "adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        else:
            optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
        return model

    def fit(self, hp, model, **kwargs):
        return model.fit(
            batch_size=hp.Int("batch_size"), shuffle=hp.Boolean("shuffle"), **kwargs
        )


tuner = RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=5,
    executions_per_trial=3,
    directory="my_dir",
    project_name="helloworld",
)


In [9]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

layer = Normalization(input_shape=(20,))
layer.adapt(x_train)

model = tf.keras.Sequential([layer, tf.keras.layers.Dense(1)])
model.compile(optimizer="adam", loss="mse")
model.fit(x_train, y_train)


4/4 [==============================] - 0s 4ms/step - loss: 2.4445


In [10]:
normalized_x_train = layer(x_train)
dataset_x_train = tf.data.Dataset.from_tensor_slices(x_train).batch(32)
normalized_dataset = dataset_x_train.map(layer)


In [11]:
from keras_tuner import HyperModel


class Regressor(HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(1))
        model.compile(optimizer="adam", loss="mse")
        return model

    def fit(self, hp, model, x, y, **kwargs):
        if hp.Boolean("normalize"):
            layer = Normalization(input_shape=(20,))
            layer.adapt(x)
            x = layer(x)
        return model.fit(x=x, y=y, **kwargs)



In [12]:
tuner = RandomSearch(Regressor(), objective="val_loss", max_trials=2)
tuner.search(x_train, y_train, validation_data=(x_val, y_val))


Trial 2 Complete [00h 00m 01s]
val_loss: 0.11157049238681793

Best val_loss So Far: 0.11157049238681793
Total elapsed time: 00h 00m 02s
